In [2]:
import pandas as pd
from scipy.spatial import cKDTree
import geopy.distance

In [3]:
def ckdnearest_brick_kilns(brick_kilns):
    A = brick_kilns[['lat', 'lon']].values
    ckd_tree = cKDTree(A)
    dist, idx = ckd_tree.query(A, k=2)
    distances = []
    for i in range(len(A)):
        point1 = (A[i, 0], A[i, 1])
        point2 = (A[idx[i, 1], 0], A[idx[i, 1], 1])
        geo_distance = geopy.distance.distance(point1, point2).km
        distances.append(geo_distance)
    closest_kiln_point = A[idx[:, 1]]
    gdf = pd.concat(
        [brick_kilns,
            pd.DataFrame(closest_kiln_point, columns=['closest_kiln_long', 'closest_kiln_lat']),
            pd.Series(distances, name='dist')], axis=1)
    return gdf

In [10]:
brick_kilns_paths = {
        'All' : '/home/shataxi.dubey/shataxi_work/Monitoring_Brick_Kiln_Compliance/combined_file.csv',
        "Punjab": '/home/shataxi.dubey/shataxi_work/Monitoring_Brick_Kiln_Compliance/punjab.csv',
        "Haryana": '/home/shataxi.dubey/shataxi_work/Monitoring_Brick_Kiln_Compliance/haryana.csv',
        "Bihar": '/home/shataxi.dubey/shataxi_work/Monitoring_Brick_Kiln_Compliance/bihar.csv',
        "Uttar Pradesh": '/home/shataxi.dubey/shataxi_work/Monitoring_Brick_Kiln_Compliance/uttar_pradesh.csv',
        "West Bengal": '/home/shataxi.dubey/shataxi_work/Monitoring_Brick_Kiln_Compliance/west_bengal.csv',
    }
state = 'West Bengal'
brick_kilns_path = brick_kilns_paths[state]
bk_kiln_mapping = ckdnearest_brick_kilns(pd.read_csv(brick_kilns_path))
bk_kiln_mapping.to_csv(f'./{state}_bk_bk_dist.csv')